<a href="https://colab.research.google.com/github/afnanhabib787/Artificial-Intelligence/blob/main/FIVER_TRANSFER_LEARNING_LEAF_CLASSIFICATION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
@author: Afnan Habib aka AI__Guru
"""

google_colab_flag = True # Make it False if do not intend to use Google Colab and want to train in local machine!!

# For training in Google Colab
if(google_colab_flag):
  from google.colab import drive
  drive.mount('/content/drive')
  !ls
  import sys
  root_path = '/content/' # This is the path to where in google drive the code is stored!
  sys.path.append(root_path)

# For local training
else:
  root_path = ''

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
drive  sample_data


In [ ]:
!unzip /content/drive/MyDrive/Jambolan.zip

# Add all necessary libraries!

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold
from PIL import Image
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import warnings
import os
import shutil
from PIL import ImageFile
warnings.simplefilter('error', Image.DecompressionBombWarning)
ImageFile.LOAD_TRUNCATED_IMAGES = True
from PIL import Image
Image.MAX_IMAGE_PIXELS = 1000000000
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/client/session.py:1768: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [ ]:
datasetFolderName=root_path+'Jambolan'
MODEL_FILENAME=root_path+"model_cv.h5"
sourceFiles=[]
classLabels=['diseased', 'healthy']
X=[]
Y=[]

img_rows, img_cols =  150, 150 # input image dimensions
train_path=datasetFolderName+'/train/'
validation_path=datasetFolderName+'/validation/'
test_path=datasetFolderName+'/test/'

In [ ]:
def transferBetweenFolders(source, dest, splitRate): 
    global sourceFiles
    sourceFiles=os.listdir(source)
    if(len(sourceFiles)!=0):
        transferFileNumbers=int(len(sourceFiles)*splitRate)
        transferIndex=random.sample(range(0, len(sourceFiles)), transferFileNumbers)
        for eachIndex in transferIndex:
            shutil.move(source+str(sourceFiles[eachIndex]), dest+str(sourceFiles[eachIndex]))
            print(source+str(sourceFiles[eachIndex]))
            print(dest+str(sourceFiles[eachIndex]))
    else:
        print("No file moved. Source empty!")
        
def transferAllClassBetweenFolders(source, dest, splitRate):
    for label in classLabels:
        transferBetweenFolders(datasetFolderName+'/'+source+'/'+label+'/', 
                               datasetFolderName+'/'+dest+'/'+label+'/', 
                               splitRate)

def my_metrics(y_true, y_pred):
    accuracy=accuracy_score(y_true, y_pred)
    precision=precision_score(y_true, y_pred,average='weighted')
    f1Score=f1_score(y_true, y_pred, average='weighted') 
    print("Accuracy  : {}".format(accuracy))
    print("Precision : {}".format(precision))
    print("f1Score : {}".format(f1Score))
    cm=confusion_matrix(y_true, y_pred)
    print(cm)
    return accuracy, precision, f1Score


# First, check if test folder is empty or not, if not transfer all existing files to train.

In [ ]:
transferAllClassBetweenFolders('test', 'train', 1.0)

No file moved. Source empty!
No file moved. Source empty!


# Now, split some part of train data into the test folders.

In [ ]:
transferAllClassBetweenFolders('train', 'test', 0.20)

In [ ]:
def prepareNameWithLabels(folderName):
    sourceFiles=os.listdir(datasetFolderName+'/train/'+folderName)
    for val in sourceFiles:
        X.append(val)
        for i in range(len(classLabels)):
          if(folderName==classLabels[i]):
              Y.append(i)

In [ ]:
# Organize file names and class labels in X and Y variables
for i in range(len(classLabels)):
  prepareNameWithLabels(classLabels[i])

In [ ]:
X=np.asarray(X)
Y=np.asarray(Y)

In [ ]:
print(X)
print(Y)


In [ ]:
from tensorflow.keras.applications import InceptionV3
conv_base = InceptionV3(weights='imagenet',
include_top=False,
input_shape=(150, 150, 3))

87924736/87910968 [==============================] - 0s 0us/step


In [ ]:
conv_base.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 74, 74, 32)   864         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 74, 74, 32)  96          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                       

In [ ]:
from tensorflow.keras import models
from tensorflow.keras import layers


In [ ]:
len(classLabels)

2

In [ ]:
# Note that, this model structure is a very basic one. To achieve better performance, you should change the model structure and hyperparameters according to your needs and data.
batch_size = 100
epoch=10
activationFunction='elu'
def getModel():
    model = models.Sequential()
    model.add(conv_base)
    #model.add(layers.Conv2D(32, (3, 3), activation='relu',
    #                        input_shape=(150, 150, 3)))
    #model.add(layers.MaxPooling2D((2, 2)))
    #model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    #model.add(layers.MaxPooling2D((2, 2)))
    #model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    #model.add(layers.MaxPooling2D((2, 2)))
    #model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    #model.add(layers.MaxPooling2D((2, 2)))
    #model.add(layers.Flatten())
    #model.add(layers.Dense(32, activation='relu'))
    #model.add(layers.Dense(16, activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation='relu' ))
    model.add(layers.Dense(64, activation='relu' ))
    #model.add(layers.Dense(256, activation='relu'))
    #model.add(layers.Dropout(0.5))
    #model.add(layers.Dense(16, activation='relu'))
    model.add(Dense(len(classLabels), activation='softmax')) 
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

model=getModel()

In [ ]:
print('This is the number of trainable weights before freezing the conv base:', len(model.trainable_weights))
conv_base.trainable = False
print('This is the number of trainable weights '
'after freezing the conv base:', len(model.trainable_weights))

This is the number of trainable weights before freezing the conv base: 194
This is the number of trainable weights after freezing the conv base: 6


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 3, 3, 2048)        21802784  
                                                                 
 flatten (Flatten)           (None, 18432)             0         
                                                                 
 dense (Dense)               (None, 256)               4718848   
                                                                 
 dense_1 (Dense)             (None, 64)                16448     
                                                                 
 dense_2 (Dense)             (None, 2)                 130       
                                                                 
Total params: 26,538,210
Trainable params: 4,735,426
Non-trainable params: 21,802,784
_________________________________________________________________


# **Stratified K-Fold Cross validation**

In [ ]:
# ===============Stratified K-Fold======================
skf = StratifiedKFold(n_splits=3, shuffle=True)
skf.get_n_splits(X, Y)
foldNum=0
for train_index, val_index in skf.split(X, Y):
    #First cut all images from validation to train (if any exists)
    transferAllClassBetweenFolders('validation', 'train', 1.0)
    foldNum+=1
    print("Results for fold",foldNum)
    X_train, X_val = X[train_index], X[val_index]
    Y_train, Y_val = Y[train_index], Y[val_index]
    # Move validation images of this fold from train folder to the validation folder
    for eachIndex in range(len(X_val)):
        classLabel=''
        for i in range(len(classLabels)):
          if(Y_val[eachIndex]==i):
              classLabel=classLabels[i]
        #Then, copy the validation images to the validation folder
        shutil.move(datasetFolderName+'/train/'+classLabel+'/'+X_val[eachIndex], 
                    datasetFolderName+'/validation/'+classLabel+'/'+X_val[eachIndex])
        
    train_datagen = ImageDataGenerator(
                      rescale=1./255,
                      rotation_range=40,
                      width_shift_range=0.2,
                      height_shift_range=0.2,
                      shear_range=0.2,
                      horizontal_flip=True,
                      zoom_range=0.20,
                      fill_mode="nearest")
    validation_datagen = ImageDataGenerator(rescale=1./255)
    test_datagen = ImageDataGenerator(rescale=1./255)
        
    #Start ImageClassification Model
    train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='categorical',
        subset='training')

    validation_generator = validation_datagen.flow_from_directory(
        validation_path,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode=None,  # only data, no labels
        shuffle=False)   
   
    # fit model
    history=model.fit_generator(train_generator, 
                        epochs=15)
    
    predictions = model.predict_generator(validation_generator, verbose=1)
    yPredictions = np.argmax(predictions, axis=1)
    true_classes = validation_generator.classes
    # evaluate validation performance
    print("***Performance on Validation data***")    
    valAcc, valPrec, valFScore = my_metrics(true_classes, yPredictions)

No file moved. Source empty!
No file moved. Source empty!
Results for fold 1
Found 333 images belonging to 2 classes.
Found 167 images belonging to 2 classes.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:51: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/15
4/4 [==============================] - 105s 18s/step - loss: 2.7554 - accuracy: 0.5255
Epoch 2/15
4/4 [==============================] - 91s 27s/step - loss: 0.7688 - accuracy: 0.5856
Epoch 3/15
4/4 [==============================] - 91s 21s/step - loss: 0.9850 - accuracy: 0.5616
Epoch 4/15
4/4 [==============================] - 91s 27s/step - loss: 0.7979 - accuracy: 0.5796
Epoch 5/15
4/4 [==============================] - 91s 21s/step - loss: 0.6867 - accuracy: 0.6907
Epoch 6/15
4/4 [==============================] - 90s 21s/step - loss: 0.5766 - accuracy: 0.7778
Epoch 7/15
4/4 [==============================] - 91s 21s/step - loss: 0.4467 - accuracy: 0.7778
Epoch 8/15
4/4 [==============================] - 90s 21s/step - loss: 0.3580 - accuracy: 0.8378
Epoch 9/15
4/4 [==============================] - 90s 27s/step - loss: 0.2645 - accuracy: 0.8889
Epoch 10/15
4/4 [==============================] - 90s 27s/step - loss: 0.2477 - accuracy: 0.8919
Epoch 11/15
4/4 [===========

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:53: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.


2/2 [==============================] - 47s 19s/step
***Performance on Validation data***
Accuracy  : 0.5508982035928144
Precision : 0.30348883072178995
f1Score : 0.3913716967609183
[[92  0]
 [75  0]]
/content/Jambolan/validation/diseased/0017_0305.JPG
/content/Jambolan/train/diseased/0017_0305.JPG
/content/Jambolan/validation/diseased/0017_0061.JPG
/content/Jambolan/train/diseased/0017_0061.JPG
/content/Jambolan/validation/diseased/0017_0190.JPG
/content/Jambolan/train/diseased/0017_0190.JPG
/content/Jambolan/validation/diseased/0017_0124.JPG
/content/Jambolan/train/diseased/0017_0124.JPG
/content/Jambolan/validation/diseased/0017_0058.JPG
/content/Jambolan/train/diseased/0017_0058.JPG
/content/Jambolan/validation/diseased/0017_0339.JPG
/content/Jambolan/train/diseased/0017_0339.JPG
/content/Jambolan/validation/diseased/0017_0049.JPG
/content/Jambolan/train/diseased/0017_0049.JPG
/content/Jambolan/validation/diseased/0017_0156.JPG
/content/Jambolan/train/diseased/0017_0156.JPG
/content

...

2/2 [==============================] - 44s 18s/step
***Performance on Validation data***
Accuracy  : 0.8143712574850299
Precision : 0.8361123906034087
f1Score : 0.8078750635212516
[[88  4]
 [27 48]]
/content/Jambolan/validation/diseased/0017_0107.JPG
/content/Jambolan/train/diseased/0017_0107.JPG
/content/Jambolan/validation/diseased/0017_0217.JPG
/content/Jambolan/train/diseased/0017_0217.JPG
/content/Jambolan/validation/diseased/0017_0082.JPG
/content/Jambolan/train/diseased/0017_0082.JPG
/content/Jambolan/validation/diseased/0017_0031.JPG
/content/Jambolan/train/diseased/0017_0031.JPG
/content/Jambolan/validation/diseased/0017_0131.JPG
/content/Jambolan/train/diseased/0017_0131.JPG
/content/Jambolan/validation/diseased/0017_0115.JPG
/content/Jambolan/train/diseased/0017_0115.JPG
/content/Jambolan/validation/diseased/0017_0151.JPG
/content/Jambolan/train/diseased/0017_0151.JPG
/content/Jambolan/validation/diseased/0017_0043.JPG
/content/Jambolan/train/diseased/0017_0043.JPG
/content/

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:51: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/15
4/4 [==============================] - 91s 26s/step - loss: 0.1717 - accuracy: 0.9341
Epoch 2/15
4/4 [==============================] - 91s 21s/step - loss: 0.0948 - accuracy: 0.9611
Epoch 3/15
4/4 [==============================] - 91s 21s/step - loss: 0.1486 - accuracy: 0.9491
Epoch 4/15
4/4 [==============================] - 91s 21s/step - loss: 0.1116 - accuracy: 0.9641
Epoch 5/15
4/4 [==============================] - 90s 21s/step - loss: 0.0644 - accuracy: 0.9820
Epoch 6/15
4/4 [==============================] - 90s 21s/step - loss: 0.0359 - accuracy: 0.9850
Epoch 7/15
4/4 [==============================] - 91s 21s/step - loss: 0.0970 - accuracy: 0.9731
Epoch 8/15
4/4 [==============================] - 91s 21s/step - loss: 0.0459 - accuracy: 0.9850
Epoch 9/15
4/4 [==============================] - 91s 21s/step - loss: 0.0537 - accuracy: 0.9760
Epoch 10/15
4/4 [==============================] - 90s 21s/step - loss: 0.0478 - accuracy: 0.9910
Epoch 11/15
4/4 [============

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:53: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.


2/2 [==============================] - 45s 19s/step
***Performance on Validation data***
Accuracy  : 0.963855421686747
Precision : 0.9660683550528645
f1Score : 0.9636625077031626
[[92  0]
 [ 6 68]]


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 3, 3, 2048)        21802784  
                                                                 
 flatten (Flatten)           (None, 18432)             0         
                                                                 
 dense (Dense)               (None, 256)               4718848   
                                                                 
 dense_1 (Dense)             (None, 64)                16448     
                                                                 
 dense_2 (Dense)             (None, 2)                 130       
                                                                 
Total params: 26,538,210
Trainable params: 4,735,426
Non-trainable params: 21,802,784
_________________________________________________________________


# Test phase

In [ ]:
print("==============TEST RESULTS============")
test_generator = test_datagen.flow_from_directory(
        test_path,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False) 
predictions = model.predict(test_generator, verbose=1)
yPredictions = np.argmax(predictions, axis=1)
true_classes = test_generator.classes

testAcc,testPrec, testFScore = my_metrics(true_classes, yPredictions)
model.save(MODEL_FILENAME)

==============TEST RESULTS============
Found 124 images belonging to 2 classes.
2/2 [==============================] - 33s 6s/step
Accuracy  : 0.9838709677419355
Precision : 0.9843253066787825
f1Score : 0.983836832223929
[[69  0]
 [ 2 53]]
